# Example 3: Optimized hyperparameters

In this example, we optimize the period and variance of the GP prior using the evidence lower bound.

Neuron `r2405_051216b_cell1816` was recorded by Dr. Marius Bauza and the Krupic lab; They have kindly shared this data to evaluate these GP methods. Please cite their work as

> Krupic J, Bauza M, Burton S, O’Keefe J. Local transformations of the hippocampal cognitive map. Science. 2018 Mar 9;359(6380):1143-6.

## Grid search to find kernel hyperparameters with best evidence lower-bound

In [ ]:
import sys
sys.path.append('../')
from lgcpspatial.loaddata        import Dataset
from lgcpspatial.hyperparameters import gridsearch_optimize

# Load a dataset
L       = 128 # Grid size for position bins
datadir = '../example data/'
dataset = 'r2405_051216b_cell1816.mat'
data    = Dataset.from_file(datadir+dataset).prepare(L,doplot=False)

result = gridsearch_optimize(data)
bestindex,bestpars,bestresult,allresults,pargrid = result

[100,99](2.43e+01,1.05e+00) loss=-9.533342e+033

## The heuristic parameters were close to optimal in this case

***Note:*** Since each time-sample adds independent information, it is helpful to normalize the evidence lower bound (ELBO). By default, `coordinate_descent` returns the negative ELBO (sometimes called the variational free energy) in units of nats-per-dataset. We can convert this to a more useful bits-per-second by:

1. Dividing by the total number of time samples, giving nats per sample
2. Multiply by `log2(e)` to convert to bits per sample
3. Multiplying by samples/second to get bits per second

This (negative) ELBO is also missing a $\ln(y!) = \ln\Gamma(y+1)$ term from the Poisson negative log-likelihood. This doesn't affect state or hyperparameter optimization, but it's good to add it back for completeness.

In [ ]:
from lgcpspatial.lgcp2d import DiagonalFourierLowrank
from lgcpspatial.lgcp2d import coordinate_descent
from lgcpspatial.lgcp2d import lgcpregress
from scipy.special      import loggamma as lnΓ
from lgcpspatial.plot   import *

mask,mz = data.arena.mask,data.prior_mean
y,N     = data.y,data.n
Fs      = 50 # Hz

inbitspers = log2(e)*Fs
correction = average(lnΓ(data.y+1),weights=data.n)
totaltime  = sum(N)

def to_bps(vfe):
    '''
    The DiagonalFourierLowrank class returns the
    variational free energy in units of nats per dataset,
    with the constant contribution from the spikes
    removed. 
    This converts our result back to bits per spike,
    adding back this constant contribution.
    '''
    return -inbitspers*(vfe/totaltime+correction)
    
# Heuristic kernel parameters
P  = data.P
kv = data.prior_variance

P_use  = P*2
v0_use = kv
result = lgcpregress(data,v0_use,P_use)
inference_summary_plot(
    result,
    ftitle='%s: Bad parameters, ELBO=%0.3f bits/second'%\
    (dataset,to_bps(result.loss)))
show()

P_use  = P
v0_use = kv
result = lgcpregress(data,v0_use,P_use)
inference_summary_plot(
    result,
    ftitle='%s: Heuristic parameters, ELBO=%0.3f bits/second'%\
    (dataset,to_bps(result.loss)))
show()

P_use  = bestpars[0]
v0_use = kv/bestpars[1]
result = lgcpregress(data,v0_use,P_use)
inference_summary_plot(
    result,
    ftitle='%s: Optimised parameters, ELBO=%0.3f bits/second'%\
    (dataset,to_bps(result.loss)))
show()

print(P_use)
print(v0_use)